In [ ]:
from src.services.decision_tree.decision_tree import DecisionTree
from tests.test_decision_tree import graph_as_dict
import networkx as nx

graph_dict = graph_as_dict()
nodes = graph_dict['nodes']
edges = graph_dict['edges']
print("node", nodes)
print("edges", edges)
#dt = DecisionTree.from_dict(graph_dict)
#dt = graph_as_dict()
#print(dt)
# assert isinstance(dt.nx, nx.DiGraph)
G = nx.Graph()
G.add_nodes_from(nodes)

for e in edges:
    tail_node = [x for x in nodes if x.id==e.tail_id][0]
    head_node = [x for x in nodes if x.id==e.head_id][0]
    name = {'name':e.name}
    G.add_edge(tail_node, head_node, **name)
#G.add_edges_from(edges)

print(f"Number of nodes: {G.number_of_nodes()}")
print(f"Number of edges: {G.number_of_edges()}")
print("Nodes in the graph:", G.nodes())
print("Edges in the graph:", G.edges())

#DiG = nx.DiGraph([(arc.endpoint_start, arc.endpoint_end) for arc in data["edges"]])
DiG = nx.DiGraph()
for e in edges:
    tail_node = [x for x in nodes if x.id==e.tail_id][0]
    head_node = [x for x in nodes if x.id==e.head_id][0]
    name = {'name':e.name}
    DiG.add_edge(tail_node, head_node, **name)
#DiG.add_edges_from(edges)
roots = [node for node in DiG.nodes if not list(DiG.predecessors(node))]
print(f"roots: {roots}")

for node in DiG.nodes:
    print(node, "  Parents ", list(DiG.predecessors(node)) )

edges_name = nx.get_edge_attributes(DiG, "name")
print(f"edge_names: {edges_name}")
tg = nx.readwrite.json_graph.tree_data(DiG, roots[0])
print(tg)






In [ ]:
from src.services.structure_service import StructureService
from src.services.scenario_service import ScenarioService
from src.dependencies import get_async_engine
import uuid

scenario_service = ScenarioService(await get_async_engine())
structure_service = StructureService(scenario_service)

import nest_asyncio
import asyncio

# Enable nested event loops
nest_asyncio.apply()

async def run_read_influence_diagram(scenario_uuid):
    issues, edges = await structure_service.read_influence_diagram(scenario_uuid)
    return issues, edges

async def run_create_decision_tree(scenario_uuid):
    await structure_service.create_decision_tree(scenario_uuid)
    return f"Decision tree created"

scenario_uuid = uuid.uuid4()
#result = asyncio.run(run_create_decision_tree(scenario_uuid))
#issues, edges = asyncio.run(run_read_influence_diagram(scenario_uuid))
#print("issues", issues)
#print("edges", edges)
result = "HH"
print(result)

In [ ]:
from src.services.scenario_service import ScenarioService
from src.services.structure_service import StructureService
from src.dependencies import get_async_engine
from uuid import uuid4
import uuid

scenario_service = ScenarioService(await get_async_engine())
structure_service = StructureService(scenario_service)

import nest_asyncio
import asyncio

# Enable nested event loops
nest_asyncio.apply()

async def run_read_influence_diagram(scenario_uuid):
    #issues, edges = await structure_service.read_influence_diagram(scenario_uuid)
    issues, edges = await scenario_service.get_influence_diagram_data(scenario_uuid)
    return issues, edges

async def run_create_decision_tree(scenario_uuid):
    return await scenario_service.get_influence_diagram_data(scenario_uuid)
    #await structure_service.create_decision_tree(scenario_uuid)
    return f"Decision tree created"

x = 'decision_tree_test_scenario_1'
scenario_uuid = uuid.uuid5(uuid.NAMESPACE_DNS, f"{x}")
#issues, edges = asyncio.run(run_create_decision_tree(scenario_uuid))
issues, edges = asyncio.run(run_read_influence_diagram(scenario_uuid))
print("issues", issues)
print("edges", edges)

issues [IssueOutgoingDto(id=UUID('ee6b08e6-9208-58eb-ae7a-5c9a6856fe23'), scenario_id=UUID('067d889c-3dd1-5cfd-8f6c-0800cac5d722'), name='Decision Issue 1', description='fc27b26f-187e-447f-a0fc-7f047ce0c54b', order=0, type='Decision', boundary='on', node=NodeViaIssueOutgoingDto(id=UUID('ee6b08e6-9208-58eb-ae7a-5c9a6856fe23'), scenario_id=UUID('067d889c-3dd1-5cfd-8f6c-0800cac5d722'), issue_id=UUID('ee6b08e6-9208-58eb-ae7a-5c9a6856fe23'), name='Decision Issue 1', node_style=NodeStyleOutgoingDto(id=UUID('ee6b08e6-9208-58eb-ae7a-5c9a6856fe23'), node_id=UUID('ee6b08e6-9208-58eb-ae7a-5c9a6856fe23'), x_position=0, y_position=0)), decision=DecisionOutgoingDto(id=UUID('ee6b08e6-9208-58eb-ae7a-5c9a6856fe23'), issue_id=UUID('ee6b08e6-9208-58eb-ae7a-5c9a6856fe23'), options=[OptionOutgoingDto(id=UUID('dda8bf80-90ac-404b-a99a-049c8105afe9'), name='yes', decision_id=UUID('ee6b08e6-9208-58eb-ae7a-5c9a6856fe23'), utility=10.0), OptionOutgoingDto(id=UUID('1ac953e7-1ae9-4a17-9795-4660e5a9482c'), name='no

In [3]:
import networkx as nx
from networkx.readwrite import json_graph

class TreeNodeDTO:
    def __init__(self, id, data, children=None):
        self.id = id
        self.data = data
        self.children = children if children is not None else []

    def __repr__(self):
        return f'TreeNodeDTO(id={self.id}, data={self.data}, children={self.children})'
    
def create_dtos_from_tree(tree_data):
    # Base case: if the tree data is empty, return None
    if not tree_data:
        return None

    # Create the DTO for the current node
    node_id = tree_data['id']
    node_data = tree_data.get('data', {})  # Use .get() to avoid KeyError
    dto = TreeNodeDTO(id=node_id, data=node_data)

    # Recursively create DTOs for child nodes
    for child in tree_data.get('children', []):
        child_dto = create_dtos_from_tree(child)
        if child_dto:
            dto.children.append(child_dto)

    return dto

G = nx.DiGraph()
G.add_edges_from([(1, 2), (1, 3), (2, 4), (2, 5)])

# Convert the graph to tree data format
tree_data = json_graph.tree_data(G, root=1)

# Create DTOs from the tree structure
dtos = create_dtos_from_tree(tree_data)

# Print the resulting DTO structure
print(dtos)

TreeNodeDTO(id=1, data={}, children=[TreeNodeDTO(id=2, data={}, children=[TreeNodeDTO(id=4, data={}, children=[]), TreeNodeDTO(id=5, data={}, children=[])]), TreeNodeDTO(id=3, data={}, children=[])])
